# CPSC 571 - Group 18

In [13]:
import pandas as pd
import numpy as np
import math

# set data frame options and program constants
SURVEYDATA = '571 Course Feedback Analysis.csv'
pd.set_option('display.max_rows', 5)
pd.set_option('display.expand_frame_repr', False)

# functions used throughout program
def calculateInfoGain(yes, no):
    totalResponses = yes + no
    yesContribution = 0
    noContribution = 0
    if yes != 0:
        yesContribution = -(yes/totalResponses) * math.log(yes/totalResponses, 2)
    if no != 0:
        noContribution = -(no/totalResponses) * math.log(no/totalResponses, 2)
    return yesContribution + noContribution

# from https://stackoverflow.com/questions/65517499/how-to-divide-dataframe-into-2-equal-parts-first-half-rows-and-second-half-rows
def split_df(df):
    if len(df) % 2 != 0:  # Handling `df` with `odd` number of rows
        df = df.iloc[:-1, :]
    df1, df2 =  np.array_split(df, 2)
    return df1, df2

# read survey data into data frame
df = pd.read_csv(SURVEYDATA)
df

,Timestamp,What Year of Study are you currently in?,How old are you?,What age did you start university?,What is your gender?,What is your faculty?,What is your major?,Are you an international student?,What is your GPA?,How many courses did you take this semester?,...,Was the course content well organized?,Were the students' questions responded to?,Was the material communicated with enthusiasm?,Were there ample opportunities for assistance?,Were students treated respectfully?,Were the evaluation methods fair?,Was the work graded in a reasonable time frame?\n,Were the support materials helpful?,Did you learn a lot in the course?,Would you recommend taking this course to another student?
0,2023/04/01 7:29:47 PM CST,Second Year,Youth (15-24 years),Youth (15-24 years),Man,Arts,Other,Yes,2 - 2.99,5,...,6,6,5,5,5,6,6,6,Yes,Yes
1,2023/04/01 7:33:47 PM CST,Upper Year,Youth (15-24 years),Youth (15-24 years),Woman,Arts,International Relations,Yes,2 - 2.99,2,...,7,7,7,7,7,6,5,7,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,2023/04/07 2:50:04 PM CST,Second Year,Youth (15-24 years),Youth (15-24 years),Man,Science,Computer Science,No,4,4,...,7,5,2,4,4,2,1,5,Yes,No
25,2023/04/08 12:09:51 PM CST,Third Year,Adult (25-64 years),Adult (25-64 years),Man,Science,Computer Science,No,3 - 3.99,5,...,6,6,6,6,7,6,7,6,Yes,Yes


In [14]:
# split data into 2 halves. one for making the model and one for the test data
modelData, testData = split_df(df)
modelData

,Timestamp,What Year of Study are you currently in?,How old are you?,What age did you start university?,What is your gender?,What is your faculty?,What is your major?,Are you an international student?,What is your GPA?,How many courses did you take this semester?,...,Was the course content well organized?,Were the students' questions responded to?,Was the material communicated with enthusiasm?,Were there ample opportunities for assistance?,Were students treated respectfully?,Were the evaluation methods fair?,Was the work graded in a reasonable time frame?\n,Were the support materials helpful?,Did you learn a lot in the course?,Would you recommend taking this course to another student?
0,2023/04/01 7:29:47 PM CST,Second Year,Youth (15-24 years),Youth (15-24 years),Man,Arts,Other,Yes,2 - 2.99,5,...,6,6,5,5,5,6,6,6,Yes,Yes
1,2023/04/01 7:33:47 PM CST,Upper Year,Youth (15-24 years),Youth (15-24 years),Woman,Arts,International Relations,Yes,2 - 2.99,2,...,7,7,7,7,7,6,5,7,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2023/04/05 10:10:24 PM CST,Upper Year,Youth (15-24 years),Youth (15-24 years),Man,Science,Actuarial Science,No,3 - 3.99,3,...,7,7,5,6,7,7,7,6,Yes,Yes
12,2023/04/06 12:44:58 PM CST,Fourth Year,Youth (15-24 years),Youth (15-24 years),Man,Science,Computer Science,Yes,Prefer not to say,6,...,3,2,4,2,6,3,6,4,Yes,Yes


In [15]:
testData

,Timestamp,What Year of Study are you currently in?,How old are you?,What age did you start university?,What is your gender?,What is your faculty?,What is your major?,Are you an international student?,What is your GPA?,How many courses did you take this semester?,...,Was the course content well organized?,Were the students' questions responded to?,Was the material communicated with enthusiasm?,Were there ample opportunities for assistance?,Were students treated respectfully?,Were the evaluation methods fair?,Was the work graded in a reasonable time frame?\n,Were the support materials helpful?,Did you learn a lot in the course?,Would you recommend taking this course to another student?
13,2023/04/06 12:48:09 PM CST,Fourth Year,Youth (15-24 years),Youth (15-24 years),Man,Science,Computer Science,Yes,Prefer not to say,6,...,6,7,7,7,7,7,7,7,Yes,No
14,2023/04/06 1:21:09 PM CST,Upper Year,Adult (25-64 years),Adult (25-64 years),Man,Science,Computer Science,No,3 - 3.99,4,...,5,6,7,7,6,7,7,6,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,2023/04/07 2:50:04 PM CST,Second Year,Youth (15-24 years),Youth (15-24 years),Man,Science,Computer Science,No,4,4,...,7,5,2,4,4,2,1,5,Yes,No
25,2023/04/08 12:09:51 PM CST,Third Year,Adult (25-64 years),Adult (25-64 years),Man,Science,Computer Science,No,3 - 3.99,5,...,6,6,6,6,7,6,7,6,Yes,Yes


# Calculate Information Gain on Attribute We Are Classifying

In [16]:
# Calculate Total Information Gain on the Classifier
WouldRecommendCounts = modelData['Would you recommend taking this course to another student? '].value_counts()
YesRecommend = WouldRecommendCounts['Yes']
NoRecommend = WouldRecommendCounts['No']
WouldRecommendInformationGain = calculateInfoGain(YesRecommend, NoRecommend)
print(f"WouldRecommendInformationGain = {WouldRecommendInformationGain}")


WouldRecommendInformationGain = 0.8904916402194913


# Calculate Info Gain for each Feature (i.e. column)

## Count Columns and their instances

In [17]:
# initialize memory data structures
finalColumn = 'Would you recommend taking this course to another student? '
counts = {}
uniquePairs = []
infoGains = {}

# loop through each column except the first (timestamp) and last (attribute we are classifying)
# for each column, we will count the number of yes/no for each different value obtained
for column_name in modelData.iloc[:, 1:-1]:

    # grab unique answers from each column (e.g. First Year, Second Year, Third Year for Year of Study)
    uniqueAnswersForGivenColumn = modelData[column_name].unique()

    # initialize memory to store counts and infoGains
    for answer in uniqueAnswersForGivenColumn:
        counts[f"{column_name}-{answer}-Yes"] = 0
        counts[f"{column_name}-{answer}-No"] = 0
        uniquePairs.append((column_name, f"{column_name}-{answer}"))
        infoGains[f"{column_name}"] = []


    print(column_name)
    for index, value in modelData.groupby([column_name, finalColumn]).size().items():
        answer = index[0]
        YesOrNo = index[1]
        counts[f"{column_name}-{answer}-{YesOrNo}"] = value
        print(f"answer = {answer}, Would you recommend taking this course to another student? {YesOrNo}, Count = {value}")
    print()


What Year of Study are you currently in?
answer = Fourth Year, Would you recommend taking this course to another student? Yes, Count = 2
answer = Second Year, Would you recommend taking this course to another student? No, Count = 2
answer = Second Year, Would you recommend taking this course to another student? Yes, Count = 3
answer = Third Year, Would you recommend taking this course to another student? No, Count = 1
answer = Upper Year, Would you recommend taking this course to another student? No, Count = 1
answer = Upper Year, Would you recommend taking this course to another student? Yes, Count = 4

How old are you?
answer = Adult (25-64 years), Would you recommend taking this course to another student? Yes, Count = 2
answer = Youth (15-24 years), Would you recommend taking this course to another student? No, Count = 4
answer = Youth (15-24 years), Would you recommend taking this course to another student? Yes, Count = 7

What age did you start university? 
answer = Adult (25-64 y

In [18]:
counts

{'What Year of Study are you currently in?-Second Year-Yes': 3,
 'What Year of Study are you currently in?-Second Year-No': 2,
 'What Year of Study are you currently in?-Upper Year-Yes': 4,
 'What Year of Study are you currently in?-Upper Year-No': 1,
 'What Year of Study are you currently in?-Third Year-Yes': 0,
 'What Year of Study are you currently in?-Third Year-No': 1,
 'What Year of Study are you currently in?-Fourth Year-Yes': 2,
 'What Year of Study are you currently in?-Fourth Year-No': 0,
 'How old are you?-Youth (15-24 years)-Yes': 7,
 'How old are you?-Youth (15-24 years)-No': 4,
 'How old are you?-Adult (25-64 years)-Yes': 2,
 'How old are you?-Adult (25-64 years)-No': 0,
 'What age did you start university? -Youth (15-24 years)-Yes': 7,
 'What age did you start university? -Youth (15-24 years)-No': 4,
 'What age did you start university? -Adult (25-64 years)-Yes': 2,
 'What age did you start university? -Adult (25-64 years)-No': 0,
 'What is your gender? -Man-Yes': 6,
 'W

In [19]:
# Calculate the Info Gain for each column's answers
# e.g. 'What Year of Study are you currently in?-Second Year-Yes': 1,
#      'What Year of Study are you currently in?-Second Year-No': 1,
# If we have for the column 'What Year of Study are you currently in?
# With a answer of Second year where one Would you recommend of Yes 1 and No 1
# Then we are calculating I(1,1) for What Year of Study Are You Currently In for the Second Year option
for pair in uniquePairs:
    column = pair[0]
    field = pair[1]
    yesCount = counts[f"{field}-Yes"]
    noCount = counts[f"{field}-No"]
    informationGain = calculateInfoGain(yesCount, noCount)
    numInstances = yesCount + noCount
    infoGains[column].append((numInstances, informationGain))

print(infoGains)





{'What Year of Study are you currently in?': [(5, 0.9709505944546686), (5, 0.7219280948873623), (1, 0.0), (2, 0.0)], 'How old are you?': [(11, 0.9456603046006402), (2, 0.0)], 'What age did you start university? ': [(11, 0.9456603046006402), (2, 0.0)], 'What is your gender? ': [(8, 0.8112781244591328), (5, 0.9709505944546686)], 'What is your faculty? ': [(2, 0.0), (1, 0.0), (9, 0.9182958340544896), (1, 0.0)], 'What is your major? ': [(2, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (1, 0.0), (6, 0.9182958340544896), (1, 0.0)], 'Are you an international student? ': [(6, 0.9182958340544896), (7, 0.863120568566631)], 'What is your GPA? ': [(4, 1.0), (5, 0.7219280948873623), (4, 0.8112781244591328)], 'How many courses did you take this semester? ': [(3, 0.9182958340544896), (1, 0.0), (3, 0.9182958340544896), (5, 0.9709505944546686), (1, 0.0)], 'Are you working part-time? ': [(6, 0.9182958340544896), (7, 0.863120568566631)], 'How many hours per day do you spend participating in studying or working on

In [20]:
# Calculate final info gain
# e.g. Gain(What Year of Study are you currently in?) = 
# = Info(Would you recommend taking this course to another student?) - Info(What Year of Study are you currently in?)
finalInfoGains = {}
for pair in uniquePairs:
    column = pair[0]
    field = pair[1]
    columnInformationGain = infoGains[column]
    infoOfColumn = 0
    for infoGain in columnInformationGain:
        numInstances = infoGain[0]
        logPart = infoGain[1]
        infoOfColumn += (numInstances/len(modelData.index)) * logPart

    gain = column
    finalGain = WouldRecommendInformationGain - infoOfColumn
    columnIndex = modelData.columns.get_loc(column)
    finalInfoGains[gain] = (finalGain, columnIndex)

finalInfoGains

{'What Year of Study are you currently in?': (0.23938445201101788, 1),
 'How old are you?': (0.09031753632664197, 2),
 'What age did you start university? ': (0.09031753632664197, 3),
 'What is your gender? ': (0.01780102730053701, 4),
 'What is your faculty? ': (0.2547483704894601, 5),
 'What is your major? ': (0.46666279373280384, 6),
 'Are you an international student? ': (0.0019055645046179848, 7),
 'What is your GPA? ': (0.05551064235231107, 8),
 'How many courses did you take this semester? ': (0.09322025740408513, 9),
 'Are you working part-time? ': (0.0019055645046179848, 10),
 'How many hours per day do you spend participating in studying or working on projects/assignments? ': (0.17932914038591197,
  11),
 'Do you have a family member or close friend attending any classes the same time as you? ': (0.025482986539758112,
  12),
 'Have you ever attended your professor’s office hours? ': (0.0019055645046179848,
  13),
 'Have you required an extension for assignment/project deadlin

# Sort info gains by highest info gain and then by order in the survey

In [21]:
sortedInfoGains = sorted(finalInfoGains.items(), key=lambda x: (x[1][0] * -1, x[1][1]))
result = []
for pair in sortedInfoGains:
    result.append((pair[0], pair[1][0]))

sortedInfoGains

[('What is the course number? (e.g. 200, 371, 599.2 etc.)',
  (0.8904916402194913, 16)),
 ('Who was the professor? ', (0.8904916402194913, 17)),
 ('Was the course content well organized? ', (0.8904916402194913, 38)),
 ('Were the support materials helpful?', (0.8904916402194913, 45)),
 ('Do you feel that the course related material/notes/resources provided by the professor were adequate? ',
  (0.6785772169761476, 30)),
 ('Was the course easier or harder than you had expected? ',
  (0.6408676019243735, 32)),
 ('Were students treated respectfully?', (0.5718986055982381, 42)),
 ('Did you learn a lot in the course?', (0.5297257989969673, 46)),
 ('What grade do you expect to receive/have received in the course? ',
  (0.5247310631299937, 31)),
 ('Was there enough detail in the course outline?', (0.5247310631299937, 36)),
 ('What is your major? ', (0.46666279373280384, 6)),
 ('What course level was it? ', (0.46666279373280384, 24)),
 ('What is the course program for the course you providing fe

# Print Calculated Information Gains

In [22]:
finalDf = pd.DataFrame(result, columns =['gain', 'value'])
finalDf


,gain,value
0,"What is the course number? (e.g. 200, 371, 599...",0.890492
1,Who was the professor?,0.890492
...,...,...
44,Please choose number 15 so that we know you ar...,0.000000
45,Have you done this course prior to this semest...,0.000000


# Write Information Gains to CSV

In [23]:
finalDf.to_csv('InformationGains.csv', encoding='utf-8', index=False)